<a href="https://colab.research.google.com/github/leomensah/leomensah.github.io/blob/main/CLASSIFICATION_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **PREPARE DATASET FOR CLASSIFICATION MODEL**

***EFFICIENT NETWORK MODEL --> FROM GOOGLE DEVELOPERS***
1. Depth Scaling

> RESNET helps in scaling the depth of the model by using a technique
called skip connections.

2. Width Scaling

> Width scaling is changing the number of feature map, thus the number of feature maps

3. Resolution Scaling

> Resolution scaling is increasing the number of pixels.

In order to pursue better accuracy, it is critical to balance all dimensions of network width, depth, and resolution during scaling.

## **Compound scaling**


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy import ndimage as ndi
from scipy import signal
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import image

In [ ]:
from sklearn.datasets import load_sample_image
from sklearn.feature_extraction import image
one_image = load_sample_image("china.jpg")
print('Image shape: {}'.format(one_image.shape))

Image shape: (427, 640, 3)


In [ ]:
def get_path_label(meta, is_clean=False):
    """This returns the path of image and mask and label list """
    image_path = list(meta['original_image'])
    mask_path = list(meta['mask_image'])
    if is_clean:
        label = [0 for x in range(len(image_path))]
    else:
        label = list(meta['is_cancer'].apply(lambda x: 1 if x=='True' else 0))
    return image_path,mask_path,label

In [ ]:
def crop_clean_patch(clean_image,dim=224):
    """Crop random patch size of dim from clean dataset"""
    clean_image = clean_image[100:400,100:400]
    for i in range(100):  
        patch = image.extract_patches_2d(clean_image,(dim,dim), max_patches=1)
        patch = np.squeeze(patch)
        if np.sum(patch)> 2000:
            return patch
    return patch

In [ ]:
def crop_nodule(coord,image,dim=112):
  x_coord = int(coord[1])+ dim
  y_coord = int(coord[0])+ dim
  image_pad = np.pad(image, ((dim,dim),(dim,dim)), 'constant', constant_values=0)
  return image_pad[y_coord-dim:y_coord+dim,x_coord-dim:x_coord+dim]

In [ ]:
def gradient_transform(patch):
  xder = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
  yder = np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
  arrx = signal.convolve2d(patch,xder,mode='same')
  arry = signal.convolve2d(patch,yder,mode='same')
  return np.hypot(arrx,arry)

In [ ]:
def laplacian_transform(patch):
    xder = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
    yder = np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
    arrx = signal.convolve2d(patch,xder,mode='same')
    arry = signal.convolve2d(patch,yder,mode='same')
    arrx = signal.convolve2d(arrx,yder,mode='same')
    arry = signal.convolve2d(arry,xder,mode='same')
    return np.hypot(arrx,arry)

In [ ]:
def create_3channel(coord,image):
    patch = crop_nodule(coord,image,dim=112)
    grad_patch = gradient_transform(patch)
    lap_patch = laplacian_transform(patch)
    output = np.stack([patch,grad_patch,lap_patch],axis=2)
    return output

In [ ]:
def create_3channel_clean(image):
  patch = crop_clean_patch(image)
  grad_patch = gradient_transform(patch)
  lap_patch = laplacian_transform(patch)
  output = np.stack([patch,grad_patch,lap_patch],axis=2)
  return output

In [ ]:
IMAGE_DIR = '/content/drive/MyDrive/data/Nodule_data/image/'
MASK_DIR = '/content/drive/MyDrive/data/Nodule_data/mask/'

# Directory to load data without any nodules, Thus, a clean lung image
CLEAN_DIR_IMG ='/content/drive/MyDrive/Clean/image/'

# Directory to save data
train_output_rgb_dir = '/content/drive/MyDrive/data/Efficient_net/train/'
val_output_rgb_dir = '/content/drive/MyDrive/data/Efficient_net/val/'
test_output_rgb_dir = '/content/drive/MyDrive/data/Efficient_net/test/'
data_label ='/content/drive/MyDrive/data/Efficient_net/label/'

# Directory to save clean data
clean_train_output_rgb_dir = '/content/drive/MyDrive/data/Efficient_net/clean_train/'
clean_val_output_rgb_dir = '/content/drive/MyDrive/data/Efficient_net/clean_val/'
clean_test_output_rgb_dir = '/content/drive/MyDrive/data/Efficient_net/clean_test/'

#Meta Information
meta = pd.read_csv('/content/drive/MyDrive/data/Meta_Beta/meta.csv')

#Clean Meta Information
clean_meta = pd.read_csv('/content/drive/MyDrive/data/Meta_Beta/clean_meta.csv')

# Get train/test label from meta.csv
meta['original_image']= meta['original_image'].apply(lambda x:IMAGE_DIR + x +'.npy')
meta['mask_image'] = meta['mask_image'].apply(lambda x:MASK_DIR + x +'.npy')

# Get train/test label from meta.csv
clean_meta['original_image']= clean_meta['original_image'].apply(lambda x:CLEAN_DIR_IMG + x +'.npy')

# Get images that were used to train Segmentation model and that is also not labeled as Ambiguous
train_meta = meta[(meta['data_split']=='Train') & (meta['is_cancer']!='Ambiguous')]
val_meta = meta[(meta['data_split']=='Validation') & (meta['is_cancer']!='Ambiguous')]
test_meta = meta[(meta['data_split']=='Test') & (meta['is_cancer']!='Ambiguous')]

# Get clean images that were used to train Segmentation model and that is also not labeled as Ambiguous
clean_train_meta = clean_meta[(clean_meta['data_split']=='Train')]
clean_val_meta = clean_meta[(clean_meta['data_split']=='Validation')]
clean_test_meta = clean_meta[(clean_meta['data_split']=='Test')]

train_image_paths,train_mask_paths, train_label  = get_path_label(train_meta)
val_image_paths, val_mask_paths, val_label = get_path_label(val_meta)
test_image_paths, test_mask_paths, test_label = get_path_label(test_meta)

clean_train_image_paths,_ ,clean_train_label  = get_path_label(clean_train_meta,True)
clean_val_image_paths,_ ,clean_val_label  = get_path_label(clean_val_meta,True)
clean_test_image_paths,_ ,clean_test_label  = get_path_label(clean_test_meta,True)

print("*"*50)
print("The length of image are train: {} test: {}".format(len(train_image_paths),len(test_image_paths)))

 #load train and save as 3 channel file
for train_img,train_mask in zip(train_image_paths,train_mask_paths):
  naming = train_img[-23:]
  mask = np.load(train_mask)
  image = np.load(train_img)
  rgb= create_3channel(ndi.center_of_mass(mask),image)
  print("Saved {}".format(naming))
  np.save(train_output_rgb_dir+naming,rgb)

#load validation and save as 3 channel file
for val_img,val_mask in zip(val_image_paths,val_mask_paths):
  naming = val_img[-23:]
  mask = np.load(val_mask)
  image = np.load(val_img)
  rgb= create_3channel(ndi.center_of_mass(mask),image)
  print("Saved {}".format(naming))
  np.save(val_output_rgb_dir+naming,rgb)

for test_img,test_mask in zip(test_image_paths,test_mask_paths):
  naming = test_img[-23:]
  mask = np.load(test_mask)
  image = np.load(test_img)
  rgb= create_3channel(ndi.center_of_mass(mask),image)
  print("Saved {}".format(naming))
  np.save(test_output_rgb_dir+naming,rgb)

In [ ]:
from sklearn.datasets import load_sample_image
from sklearn.feature_extraction import image
one_image = load_sample_image("china.jpg")
print(type(one_image))

<class 'numpy.ndarray'>


In [ ]:
for train_img in clean_train_image_paths[:800]:
    naming = train_img[-23:]
    new_image = np.load(train_img)
    clean_image = new_image[100:400,100:400]
    print(type(clean_image))
    print(clean_image.shape)
    patches = image.extract_patches_2d(clean_image, (224, 224))
    # patch = image.extract_patches_2d(clean_image,(224,224),1)
    # for i in range(100):  
        #patch = image.extract_patches_2d(clean_image,(224,224),1)
        # patch = np.squeeze(patch)
        # if np.sum(patch)> 2000:
        #     return patch
    print(patches)
    print(type(clean_image))
    break
    rgb = create_3channel_clean(image)

In [ ]:
# Converting the clean dataset to rgb and training
# and saving the output
for train_img in clean_train_image_paths[:800]:
    naming = train_img[-23:]
    new_image = np.load(train_img)
    rgb = create_3channel_clean(new_image)
    print("Saved {}".format(naming))
    np.save(clean_train_output_rgb_dir+naming,rgb)

for val_img in clean_val_image_paths[:400]:
    naming = val_img[-23:]
    new_image = np.load(val_img)
    rgb = create_3channel_clean(new_image)
    print("Saved {}".format(naming))
    #np.save(clean_val_output_rgb_dir+naming,rgb)

for test_img in clean_test_image_paths[:400]:
    naming = test_img[-23:]
    new_image = np.load(test_img)
    rgb = create_3channel_clean(new_image)
    print("Saved {}".format(naming))
    np.save(clean_test_output_rgb_dir+naming,rgb)

Saved 0028_CN001_slice000.npy
Saved 0028_CN001_slice001.npy
Saved 0028_CN001_slice002.npy
Saved 0028_CN001_slice003.npy
Saved 0028_CN001_slice004.npy
Saved 0028_CN001_slice005.npy
Saved 0028_CN001_slice006.npy
Saved 0028_CN001_slice007.npy
Saved 0028_CN001_slice008.npy
Saved 0028_CN001_slice009.npy
Saved 0028_CN001_slice010.npy
Saved 0028_CN001_slice011.npy
Saved 0028_CN001_slice012.npy
Saved 0028_CN001_slice013.npy
Saved 0028_CN001_slice014.npy
Saved 0028_CN001_slice015.npy
Saved 0028_CN001_slice016.npy
Saved 0028_CN001_slice017.npy
Saved 0028_CN001_slice018.npy
Saved 0028_CN001_slice019.npy
Saved 0028_CN001_slice020.npy
Saved 0028_CN001_slice021.npy
Saved 0028_CN001_slice022.npy
Saved 0028_CN001_slice023.npy
Saved 0028_CN001_slice024.npy
Saved 0028_CN001_slice025.npy
Saved 0028_CN001_slice026.npy
Saved 0028_CN001_slice027.npy
Saved 0028_CN001_slice028.npy
Saved 0028_CN001_slice029.npy
Saved 0028_CN001_slice030.npy
Saved 0028_CN001_slice031.npy
Saved 0028_CN001_slice032.npy
Saved 0028

In [ ]:
# with open(data_label+'train.txt','wb') as fp:
#     pickle.dump(train_label,fp)
# with open(data_label+'val.txt','wb') as fp:
#     pickle.dump(val_label,fp)
# with open(data_label+'test.txt','wb') as fp:
#     pickle.dump(test_label,fp)

In [ ]:
# clean_train_label = clean_train_label[:800]
# clean_val_label = clean_val_label[:400]
# clean_test_label = clean_test_label[:400]

# with open(data_label+'clean_train.txt','wb') as fp:
#     pickle.dump(clean_train_label,fp)
# with open(data_label+'clean_val.txt','wb') as fp:
#     pickle.dump(clean_val_label,fp)
# with open(data_label+'clean_test.txt','wb') as fp:
#     pickle.dump(clean_test_label,fp)      

In [ ]:
# print("TOTAL OF CANCER: {}, NON-CANCEROUS:{} IMAGES WERE SAVED FOR TRAIN".format(np.sum(train_label),len(train_label)-np.sum(train_label)))
# print("TOTAL OF CANCER: {}, NON-CANCEROUS:{} IMAGES WERE SAVED FOR VAL".format(np.sum(val_label),len(val_label)-np.sum(val_label)))
# print("TOTAL OF CANCER: {}, NON-CANCEROUS:{} IMAGES WERE SAVED FOR TEST".format(np.sum(test_label),len(test_label)-np.sum(test_label)))

# print("AS THE DATA IS IMBALANCED, WE ADDED CLEAN IMAGES AS FOLLOWING")
# print("TRAIN: {}, VAL: {}, TEST: {}".format(len(clean_train_label),len(clean_val_label),len(clean_test_label)))

TOTAL OF CANCER: 3146, NON-CANCEROUS:1974 IMAGES WERE SAVED FOR TRAIN
TOTAL OF CANCER: 914, NON-CANCEROUS:728 IMAGES WERE SAVED FOR VAL
TOTAL OF CANCER: 1189, NON-CANCEROUS:572 IMAGES WERE SAVED FOR TEST
AS THE DATA IS IMBALANCED, WE ADDED CLEAN IMAGES AS FOLLOWING
TRAIN: 800, VAL: 400, TEST: 400


### **CLASSIFIER DATASET**

In [ ]:
!pip install albumentations==0.4.6

     |████████████████████████████████| 117 kB 4.3 MB/s 
     |████████████████████████████████| 948 kB 29.3 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65174 sha256=25411a18a48d0c2a6c2147d935fa2a5300306a2a9af5411d1d0ed2387cd77e62
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
import os
import numpy as np
import glob
from PIL import Image

import torch
from torch.utils.data.dataset import Dataset
import torchvision.transforms.functional as TF
import torchvision
from torchvision import transforms

import albumentations as albu
from albumentations.pytorch import ToTensorV2
from albumentations import OneOf,Compose

class ClassifierDataset(Dataset):
    def __init__(self, IMAGES_PATHS,label,Albumentation=False):
      self.image_paths = IMAGES_PATHS
      self.labels = label
      self.albumentation = Albumentation

      self.albu_transformations =  albu.Compose([
          albu.Normalize(),
          ToTensorV2(),
          OneOf([albu.HorizontalFlip(),
                  albu.VerticalFlip(),
                  albu.RandomRotate90(),
                  ],p=0.9)
          #albu.ElasticTransform(alpha=1.1,alpha_affine=0.5,sigma=5,p=0.15),
      ])
      self.transformations = transforms.Compose([
          #transforms.Grayscale(3),
          transforms.ToTensor(),
          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def transform(self, image):
      #Transform to tensor
      if self.albumentation:
          #It is always best to convert the make input to 3 dimensional for albumentation
          augmented=  self.albu_transformations(image=image)
          image = augmented['image']
      else:
          image = self.transformations(image)
      image= image.type(torch.FloatTensor),
      return image

    def __getitem__(self, index):
      image = np.load(self.image_paths[index])
      image = self.transform(image)
      return image[0],self.labels[index]

    def __len__(self):
      return len(self.image_paths)


### **UTILS**

In [ ]:
import argparse
import os

def str2bool(v):
    if v.lower() in ['true', 1]:
        return True
    elif v.lower() in ['false', 0]:
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def load_directories(root_dir):
    images_list = os.listdir(root_dir)
    images_list.sort()
    return [root_dir+ x for x in images_list]

## **METRICS**

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F

def Accuracy(output, label):
  total = 0
  correct =0
  output_prob = (output>0.5).int()
  correct += torch.sum(output_prob==label).item()
  total += len(label)

  return  correct / total

def Confusion_matrix(output, label):
  total = 0
  smooth = 0.1

  output_prob = (output>0.5).int()
  label = label.int()

  conf_matrix = torch.zeros(2, 2)
  for t, p in zip(label, output_prob):
      conf_matrix[t, p] += 1
  TP = conf_matrix[1,1].item()
  TN = conf_matrix[0,0].item()
  FP = conf_matrix[0,1].item()
  FN = conf_matrix[1,0].item()

  total += len(label)
  accuracy = (TP+TN)/total
  sensitivity = TP / (TP+FN+ smooth)
  specificity = TN / (TN+FP+ smooth)
  
  return  accuracy, sensitivity, specificity

def Confusion_matrix2(output, label):
  total = 0
  smooth = 0.1

  output_prob = (output>0.5).int()
  label = label.int()

  conf_matrix = torch.zeros(2, 2)
  for t, p in zip(label, output_prob):
      conf_matrix[t, p] += 1
  TP = conf_matrix[1,1].item()
  TN = conf_matrix[0,0].item()
  FP = conf_matrix[0,1].item()
  FN = conf_matrix[1,0].item()

  return  TP,TN,FN,FP

### **TRAIN CLASSIFIER**

In [ ]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=3a634fedb9f3c5f7fa3b2bb46c9333b0626b2bc96cc88c99b894de0b2385eb74
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [ ]:
import pandas as pd
import numpy as np
import argparse
import os
from collections import OrderedDict
from glob import glob
import pickle
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import yaml

import torch
import torch.backends.cudnn as cudnn
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

from torchvision import datasets, models, transforms
from PIL import Image
from efficientnet_pytorch import EfficientNet

In [ ]:
def train(train_loader,model,criterion,optimizer):
    avg_meters = {'loss': AverageMeter(),
                'accuracy': AverageMeter(),
                'sensitivity':AverageMeter(),
                'specificity': AverageMeter()}

    model.train()
    pbar = tqdm(total=len(train_loader))
    for images, labels in train_loader:
        images = images.cuda()
        labels = labels.cuda()

        outputs = model(images)
        outputs = outputs.view(-1)
        labels = labels.type_as(outputs)
        loss = criterion(outputs, labels)
        accuracy, sensitivity, specificity = Confusion_matrix(outputs,labels)
        #print(loss)

        # compute gradient and do optimizing step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_meters['loss'].update(loss.item(),images.size(0))
        avg_meters['accuracy'].update(accuracy,images.size(0))
        avg_meters['sensitivity'].update(sensitivity,images.size(0))
        avg_meters['specificity'].update(specificity,images.size(0))

        postfix = OrderedDict([
            ('loss', avg_meters['loss'].avg),
            ('accuracy', avg_meters['accuracy'].avg),
            ('sensitivity', avg_meters['sensitivity'].avg),
            ('specificity', avg_meters['specificity'].avg)
        ])
        pbar.set_postfix(postfix)
        pbar.update(1)
    pbar.close()

    return OrderedDict([('loss', avg_meters['loss'].avg),
                        ('accuracy', avg_meters['accuracy'].avg),
                        ('sensitivity', avg_meters['sensitivity'].avg),
                        ('specificity', avg_meters['specificity'].avg),])

In [ ]:
def validate(val_loader,model,criterion):
    avg_meters = {'loss': AverageMeter(),
                'accuracy': AverageMeter(),
                'sensitivity':AverageMeter(),
                'specificity': AverageMeter()}
    model.eval()

    pbar = tqdm(total=len(val_loader))
    for images, labels in val_loader:
        images = images.cuda()
        labels = labels.cuda()

        outputs = model(images)
        outputs = outputs.view(-1)
        labels = labels.type_as(outputs)
        loss = criterion(outputs, labels)
        accuracy, sensitivity, specificity = Confusion_matrix(outputs,labels)

        avg_meters['loss'].update(loss.item(),images.size(0))
        avg_meters['accuracy'].update(accuracy,images.size(0))
        avg_meters['sensitivity'].update(sensitivity,images.size(0))
        avg_meters['specificity'].update(specificity,images.size(0))

        postfix = OrderedDict([
            ('loss', avg_meters['loss'].avg),
            ('accuracy', avg_meters['accuracy'].avg),
            ('sensitivity', avg_meters['sensitivity'].avg),
            ('specificity', avg_meters['specificity'].avg)
        ])

        pbar.set_postfix(postfix)
        pbar.update(1)
    pbar.close()

    return OrderedDict([('loss', avg_meters['loss'].avg),
                        ('accuracy', avg_meters['accuracy'].avg),
                        ('sensitivity', avg_meters['sensitivity'].avg),
                        ('specificity', avg_meters['specificity'].avg),])

In [ ]:
# Model Output directory
OUTPUT_DIR = '/content/drive/MyDrive/data/Classification/model_output/'
filename = 0
epochs = 200
batch_size = 24
early_stopping = 30
num_workers = 2
optimizer = 'Adam'
learning_rate = 1e-6
momentum = 0.9
weight_decay = 1e-4


os.makedirs(OUTPUT_DIR+'efficientnetb{}'.format(filename),exist_ok=True)
print('Made directory called efficientnetb{}'.format(filename))

print('-' * 20)

#save configuration
# with open(OUTPUT_DIR+'efficientnetb{}/config.yml'.format(filename), 'w') as f:
#     yaml.dump(config, f)

# Data directory
TRAIN_DIR = '/content/drive/MyDrive/data/Efficient_net/train/'
VAL_DIR = '/content/drive/MyDrive/data/Efficient_net/val/'
CLEAN_TRAIN_DIR ='/content/drive/MyDrive/data/Efficient_net/clean_train/'
CLEAN_VAL_DIR = '/content/drive/MyDrive/data/Efficient_net/clean_val/'
LABEL_DIR = '/content/drive/MyDrive/data/Efficient_net/label/'

with open(LABEL_DIR+'train.txt','rb') as fp:
  train_label = pickle.load(fp)
with open(LABEL_DIR+'val.txt','rb') as fp:
  val_label = pickle.load(fp)
with open(LABEL_DIR+'clean_train.txt','rb') as fp:
  clean_train_label = pickle.load(fp)
with open(LABEL_DIR+'clean_val.txt','rb') as fp:
  clean_val_label = pickle.load(fp)

# Get image files path as list
train_image_paths = load_directories(TRAIN_DIR)
val_image_paths = load_directories(VAL_DIR)
clean_train_images_paths = load_directories(CLEAN_TRAIN_DIR)
clean_val_images_paths = load_directories(CLEAN_VAL_DIR)

train_image_paths.extend(clean_train_images_paths)
val_image_paths.extend(clean_val_images_paths)
train_label.extend(clean_train_label)
val_label.extend(clean_val_label)

print("="*50)
print("The length of image are train: {} validation: {}".format(len(train_image_paths),len(val_image_paths)))

print("============================TRAINING===========================================")
print("Cancer nodules:{} Non Cancer nodules:{}".format(np.sum(train_label),len(train_label)-np.sum(train_label)))
print("Ratio is {:4f}".format(np.sum(train_label)/(len(train_label)-np.sum(train_label))))
print("============================VALIDATION=========================================")
print("Cancer nodules:{} Non Cancer nodules:{}".format(np.sum(val_label),len(val_label)-np.sum(val_label)))
print("Ratio is {:4f}".format(np.sum(val_label)/(len(val_label)-np.sum(val_label))))
# Create Dataset
train_dataset = ClassifierDataset(train_image_paths,train_label)
val_dataset = ClassifierDataset(val_image_paths,val_label)

# Model
cudnn.benchmark = True
model = EfficientNet.from_pretrained('efficientnet-b{}'.format(filename))

#Fine tuning top layers
num_ftrs = model._fc.in_features
model._fc = nn.Sequential(nn.Linear(num_ftrs,1),
                            nn.Sigmoid())

criterion = nn.BCEWithLogitsLoss().cuda()


if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
model = model.cuda()
params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = optim.Adam(params, lr=learning_rate, weight_decay=weight_decay)

#if config['optimizer'] == 'Adam':
#elif config['optimizer'] == 'SGD':
#    optimizer = optim.SGD(params, lr=config['lr'], momentum=config['momentum'],nesterov=config['nesterov'], weight_decay=config['weight_decay'])
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
# Create Dataloader
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    pin_memory=True,
    drop_last=True,
    num_workers=6)
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    pin_memory=True,
    drop_last=False,
    num_workers=6)

log= pd.DataFrame(index=[],columns= ['epoch', 'loss', 'accuracy','sensitivity','specificity,',
                                    'val_loss', 'val_accuracy','val_sensitivity','val_specificity'])

best_loss= 10
trigger = 0

for epoch in range(epochs):

    # train for one epoch
    train_log = train(train_loader, model, criterion, optimizer)
    # evaluate on validation set
    val_log = validate(val_loader, model, criterion)

    print('Training epoch [{}/{}], Training BCE loss:{:.4f}, Training accuracy:{:.4f}, Training sensitivity:{:.4f}, Training specificity:{:.4f}, \
                Validation BCE loss:{:.4f}, Validation accuracy:{:.4f}, Validation sensitivity:{:.4f}, Validation specificity:{:.4f},'.format( 
        epoch + 1, epochs, train_log['loss'], train_log['accuracy'], train_log['sensitivity'], train_log['specificity'],val_log['loss'], val_log['accuracy'],val_log['sensitivity'], val_log['specificity']))

    tmp = pd.Series([
        epoch,
        train_log['loss'],
        train_log['accuracy'],
        train_log['sensitivity'],
        train_log['specificity'],
        val_log['loss'],
        val_log['accuracy'],
        val_log['sensitivity'],
        val_log['specificity'],
    ], index=['epoch', 'loss', 'accuracy','sensitivity','specificity,','val_loss', 'val_accuracy','val_sensitivity','val_specificity'])

    log = log.append(tmp, ignore_index=True)
    log.to_csv(OUTPUT_DIR+'efficientnetb{}/log.csv'.format(filename), index=False)

    trigger += 1

    if val_log['loss'] < best_loss:
        torch.save(model.state_dict(), OUTPUT_DIR+'efficientnetb{}/model.pth'.format(filename))
        best_loss= val_log['loss']
        print("=> saved best model as validation loss is greater than previous best loss")
        trigger = 0

    # early stopping
    if early_stopping >= 0 and trigger >= early_stopping:
        print("=> early stopping")
        break

    torch.cuda.empty_cache()

Made directory called efficientnetb0
--------------------
The length of image are train: 5920 validation: 2042
============================TRAINING===========================================
Cancer nodules:3146 Non Cancer nodules:2774
Ratio is 1.134102
============================VALIDATION=========================================
Cancer nodules:914 Non Cancer nodules:1128
Ratio is 0.810284


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth


  0%|          | 0.00/20.4M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 86/86 [02:24<00:00,  1.67s/it, loss=0.744, accuracy=0.579, sensitivity=0.239, specificity=0.624]


...

Training epoch [100/200], Training BCE loss:0.5419, Training accuracy:0.9068, Training sensitivity:0.8768, Training specificity:0.9393,                 Validation BCE loss:0.6545, Validation accuracy:0.7135, Validation sensitivity:0.4050, Validation specificity:0.6319,


100%|██████████| 86/86 [00:06<00:00, 13.96it/s, loss=0.656, accuracy=0.714, sensitivity=0.409, specificity=0.628]


Training epoch [101/200], Training BCE loss:0.5413, Training accuracy:0.9045, Training sensitivity:0.8785, Training specificity:0.9342,                 Validation BCE loss:0.6560, Validation accuracy:0.7140, Validation sensitivity:0.4091, Validation specificity:0.6281,


100%|██████████| 86/86 [00:05<00:00, 14.59it/s, loss=0.655, accuracy=0.714, sensitivity=0.402, specificity=0.633]

Training epoch [102/200], Training BCE loss:0.5407, Training accuracy:0.9072, Training sensitivity:0.8795, Training specificity:0.9389,                 Validation BCE loss:0.6550, Validation accuracy:0.7135, Validation sensitivity:0.4021, Validation specificity:0.6329,
=> early stopping


## **EVALUATE THE EFFICIENT NET MODEL**

In [ ]:
def evaluate(model,test_loader):
    with torch.no_grad():
        counter = 0
        pbar = tqdm(total=len(test_loader))
        TP,TN,FN,FP = 0,0,0,0
        for images, labels in test_loader:
            images = images.cuda()
            labels = labels.cuda()

            output = model(images)
            outputs = output.view(-1)
            labels = labels.type_as(outputs)
            TP_batch,TN_batch,FN_batch,FP_batch = Confusion_matrix2(outputs,labels)
            TP += TP_batch
            TN += TN_batch
            FN += FN_batch
            FP += FP_batch
            
            pbar.update(1)
        pbar.close()

        accuracy = (TP+TN)/ (TP +TN +FN +FP)
        sensitivity = TP / (TP+FN)
        specificity = TN / (TN+FP)
        precision = TP / (TP+FP)
        F1 = (2*sensitivity*precision)/(precision+sensitivity)
    torch.cuda.empty_cache()
    return accuracy, sensitivity, specificity, precision,F1, TP,TN,FN,FP

In [ ]:
filename = 0
epochs = 200
batch_size = 24
early_stopping = 30
num_workers = 2
optimizer = 'Adam'
learning_rate = 1e-6
momentum = 0.9
weight_decay = 1e-4

OUTPUT_DIR = '/content/drive/MyDrive/data/Classification/model_output/efficientnetb0/'
TEST_DIR = '/content/drive/MyDrive/data/Efficient_net/test/'
CLEAN_TEST_DIR = '/content/drive/MyDrive/data/Efficient_net/clean_test/'
LABEL_DIR ='/content/drive/MyDrive/data/Efficient_net/label/'

with open(LABEL_DIR+'test.txt','rb') as fp:
    test_label = pickle.load(fp)
with open(LABEL_DIR+'clean_test.txt','rb') as fp:
    clean_test_label = pickle.load(fp)

print('-'*20)

cudnn.benchmark = True
model = EfficientNet.from_pretrained('efficientnet-b{}'.format(filename))

#Fine tuning top layers
num_ftrs = model._fc.in_features
model._fc = nn.Sequential(nn.Linear(num_ftrs,1),
                            nn.Sigmoid())

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

model.load_state_dict(torch.load(OUTPUT_DIR +'model.pth'))
model = model.cuda()

# Get image files
test_image_paths = load_directories(TEST_DIR)
clean_test_image_paths = load_directories(CLEAN_TEST_DIR)

test_image_paths.extend(clean_test_image_paths)
test_label.extend(clean_test_label)

assert len(test_image_paths)==len(test_label), "Length of test images and test label not same"
print("============================TESTING===========================================")
print("Cancer nodules:{} Non Cancer nodules:{}".format(np.sum(test_label),len(test_label)-np.sum(test_label)))
print("Ratio is {:4f}".format(np.sum(test_label)/(len(test_label)-np.sum(test_label))))

test_dataset = ClassifierDataset(test_image_paths,test_label,Albumentation=False)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    pin_memory=True,
    drop_last=False,
    num_workers=6)

log= pd.DataFrame(index=[],columns= ['test_size','accuracy','sensitivity','specificity','TP','TN','FN','FP'])

accuracy, sensitivity, specificity, precision,F1, TP,TN,FN,FP = evaluate(model,test_loader)

tmp = pd.Series([
    len(test_dataset),accuracy, sensitivity, specificity, TP, TN, FN, FP
], index=['test_size','accuracy','sensitivity','specificity','TP','TN','FN','FP'])

print('Test accuracy:{:.4f}, Test sensitivity:{:.4f}, Testspecificity:{:.4f}'.format(accuracy,sensitivity,specificity))
print('Test precision:{:.4f}, Test F1:{:.4f}'.format(precision,F1))
log = log.append(tmp,ignore_index=True)
log.to_csv(OUTPUT_DIR +'test_result.csv',index=False)
print("OUTPUT RESULT SAVED AS CSV in", OUTPUT_DIR)

--------------------
Loaded pretrained weights for efficientnet-b0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


============================TESTING===========================================
Cancer nodules:1189 Non Cancer nodules:972
Ratio is 1.223251


100%|██████████| 91/91 [02:11<00:00,  1.44s/it]

Test accuracy:0.5747, Test sensitivity:0.4853, Testspecificity:0.6842
Test precision:0.6527, Test F1:0.5567
OUTPUT RESULT SAVED AS CSV in /content/drive/MyDrive/data/Classification/model_output/efficientnetb0/
